In [1]:
import os
import time
import glob
import yaml
import argparse
import logging
import pathlib
import struct
import warnings
from random import randint

import numpy as np
import torch
from torch_scatter import scatter
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull, Delaunay, cKDTree, KDTree
from sklearn.neighbors import NearestNeighbors

import open3d as o3d
import trimesh
import k3d
import wandb

from main import write_to_csv, generate_data_list, load_data

from openpoints.models import build_model_from_cfg
from openpoints.utils import (
    get_mious,
    ConfusionMatrix,
    set_random_seed,
    load_checkpoint,
    setup_logger_dist,
    cal_model_parm_nums,
    Wandb,
    generate_exp_directory,
    EasyConfig,
    dist_utils,
)
from openpoints.transforms import build_transforms_from_cfg
from openpoints.dataset import (
    build_dataloader_from_cfg,
    get_features_by_keys,
    get_class_weights,
)
from openpoints.dataset.data_util import voxelize

warnings.simplefilter(action='ignore', category=FutureWarning)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
@torch.no_grad()
def predict(model, coord, feat, idx_points, cfg, num_votes=1):
    """using a part of original point cloud as input to save memory.
    Args:
        model (_type_): _description_
        test_loader (_type_): _description_
        cfg (_type_): _description_
        num_votes (int, optional): _description_. Defaults to 1.
    Returns:
        _type_: _description_
    """
    model.eval()  # set model to eval mode

    # data
    trans_split = 'val' if cfg.datatransforms.get('test', None) is None else 'test'
    pipe_transform = build_transforms_from_cfg(trans_split, cfg.datatransforms)
    
    gravity_dim = cfg.datatransforms.kwargs.gravity_dim

    all_logits = []

    len_part = len(idx_points)
    nearest_neighbor = len_part == 1
    pbar = tqdm(range(len(idx_points)))

    for idx_subcloud in pbar:
        pbar.set_description(f"Test on [{idx_subcloud}]/[{len_part}]")

        idx_part = idx_points[idx_subcloud]
        coord_part = coord[idx_part]
        coord_part -= coord_part.min(0)

        feat_part =  feat[idx_part] if feat is not None else None
        data = {'pos': coord_part}
        if feat_part is not None:
            data['x'] = feat_part
        if pipe_transform is not None:
            data = pipe_transform(data)
        if 'heights' in cfg.feature_keys and 'heights' not in data.keys():
            if 'semantickitti' in cfg.dataset.common.NAME.lower():
                data['heights'] = torch.from_numpy((coord_part[:, gravity_dim:gravity_dim + 1] - coord_part[:, gravity_dim:gravity_dim + 1].min()).astype(np.float32)).unsqueeze(0)
            else:
                data['heights'] = torch.from_numpy(coord_part[:, gravity_dim:gravity_dim + 1].astype(np.float32)).unsqueeze(0)
        if not cfg.dataset.common.get('variable', False):
            if 'x' in data.keys():
                data['x'] = data['x'].unsqueeze(0)
            data['pos'] = data['pos'].unsqueeze(0)
        else:
            data['o'] = torch.IntTensor([len(coord)])
            data['batch'] = torch.LongTensor([0] * len(coord))

        for key in data.keys():
            data[key] = data[key].cuda(non_blocking=True)
        if 'x' in data:
            data['x'] = get_features_by_keys(data, cfg.feature_keys)
        
        logits = model(data)
        all_logits.append(logits)

    all_logits = torch.cat(all_logits, dim=0)

    if not cfg.dataset.common.get('variable', False):
        all_logits = all_logits.transpose(1, 2).reshape(-1, cfg.num_classes)
    
    if not nearest_neighbor:
        # average merge overlapped multi voxels logits to original point set
        idx_points = torch.from_numpy(np.hstack(idx_points)).cuda(non_blocking=True)
        all_logits = scatter(all_logits, idx_points, dim=0, reduce='mean')

    pred = all_logits.argmax(dim=1)
    
    return pred



In [3]:
parser = argparse.ArgumentParser('S3DIS scene segmentation training')
cfg = EasyConfig()
_, opts = parser.parse_known_args()
cfg_path = "/scratch/rhm4nj/cral/cral-ginn/ginn/pointnext/cfgs/s3dis/pointnext-l.yaml"
cfg.load(cfg_path, recursive=True)
cfg.update(opts)

cfg.pretrained_path = "/scratch/rhm4nj/cral/cral-ginn/ginn/pointnext/models/s3dis-train-pointnext-l-ngpus1-seed6266-20220525-162629-D7sCFuHmsMP9Kk5bdAA2Td_ckpt_best.pth"
# cfg.dataset.common.data_root = "/scratch/rhm4nj/cral/datasets/Stanford3dDataset_v1.2"
data_path = "/scratch/rhm4nj/cral/datasets/Stanford3dDataset_v1.2/raw/Area_1.txt"
# data_path = "/scratch/rhm4nj/cral/datasets/Stanford3dDataset_v1.2/Area_1/office_1/office_1.txt"

cfg.seed = 10
cfg.rank, cfg.world_size, cfg.distributed, cfg.mp = dist_utils.get_dist_info(cfg)
cfg.mode = 'test'
cfg.is_training = False

# [0, 1, 2, 3, 6, 8]
# [ 0  1  2  3  6  7  8 10 11 12]
cfg.classes = ['ceiling',
                'floor',
                'wall',
                'beam',
                'column',
                'window',
                'door',
                'chair',
                'table',
                'bookcase',
                'sofa',
                'board',
                'clutter']

model = build_model_from_cfg(cfg.model).to(cfg.rank)
best_epoch, best_val = load_checkpoint(
    model, pretrained_path=cfg.pretrained_path)

data = np.loadtxt(data_path)
coord, feat = data[:, :3], data[:, 3:6]
feat = np.clip(feat / 255., 0, 1).astype(np.float32)

idx_points = []
voxel_idx, reverse_idx_part,reverse_idx_sort = None, None, None
voxel_size = cfg.dataset.common.get('voxel_size', None)

if voxel_size is not None:
    # idx_sort: original point indicies sorted by voxel NO.
    # voxel_idx: Voxel NO. for the sorted points
    idx_sort, voxel_idx, count = voxelize(coord, voxel_size, mode=1)
    if cfg.get('test_mode', 'multi_voxel') == 'nearest_neighbor':
        idx_select = np.cumsum(np.insert(count, 0, 0)[0:-1]) + np.random.randint(0, count.max(), count.size) % count
        idx_part = idx_sort[idx_select]
        npoints_subcloud = voxel_idx.max()+1
        idx_shuffle = np.random.permutation(npoints_subcloud)
        idx_part = idx_part[idx_shuffle] # idx_part: randomly sampled points of a voxel
        reverse_idx_part = np.argsort(idx_shuffle, axis=0) # revevers idx_part to sorted
        idx_points.append(idx_part)
        reverse_idx_sort = np.argsort(idx_sort, axis=0)
    else:
        for i in range(count.max()):
            idx_select = np.cumsum(np.insert(count, 0, 0)[0:-1]) + i % count
            idx_part = idx_sort[idx_select]
            np.random.shuffle(idx_part)
            idx_points.append(idx_part)
else:
    idx_points.append(np.arange(coord.shape[0]))

# print(coord.shape, len(idx_points), feat.shape)

pred = predict(model, coord, feat, idx_points, cfg, num_votes=1)


launch mp with 3 GPUs, current rank: 0


Test on [52]/[53]: 100%|██████████| 53/53 [00:27<00:00,  1.96it/s]


In [4]:
# import k3d

# print(pred.shape)

# colormap = [
#     0xFF0000, 0x00FF00, 0x0000FF, 0xFFFF00, 0xFF00FF, 0x00FFFF, 0xFFFFFF, 
#     0xFF8000, 0x8000FF, 0x0080FF, 0x80FF00, 0xFF0080, 0x808080
# ]

# color_names = [
#     "Red", "Green", "Blue", "Yellow", "Magenta", "Cyan", "White",
#     "Orange", "Purple", "Sky Blue", "Lime Green", "Pink", "Gray"
# ]

# colors = np.array([colormap[int(label) % len(colormap)] for label in pred], dtype=np.uint32)

# plot = k3d.plot()
# points = k3d.points(coord, point_size=0.1, colors=colors)

# plot += points
# plot.display()

In [5]:
import numpy as np
from sklearn.cluster import DBSCAN
import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN

flat_classes = ['ceiling','floor','wall', 'beam', 'door', 'chair']
flats = []

print([cfg.classes.index(c) for c in flat_classes])

color_names = [
    "Red", "Green", "Blue", "Yellow", "Magenta", "Cyan", "White",
    "Orange", "Purple", "Sky Blue", "Lime Green", "Pink", "Gray"
]

def get_point_cloud_bounds(point_cloud):
    min_vals = np.min(point_cloud, axis=0)
    max_vals = np.max(point_cloud, axis=0)

    return [[min_vals[0], max_vals[0]],  # x_min, x_max
            [min_vals[1], max_vals[1]],  # y_min, y_max
            [min_vals[2], max_vals[2]]]  # z_min, z_max

def too_small(c, thresh=0.01):
    bounds = get_point_cloud_bounds(c)
    (xmin, xmax), (ymin, ymax), (zmin, zmax) = bounds
    volume = (xmax - xmin) * (ymax - ymin) * (zmax - zmin)
    return volume < 0.05

def segment_individual(point_cloud, labels, eps=0.035, min_samples=5):
    unique_classes = np.unique(labels)
    print(unique_classes)
    segmented_objects = []

    # plot = k3d.plot()
    # plot.display()

    for class_label in unique_classes:
        # Extract points of the current class
        class_mask = labels == class_label
        class_points = point_cloud[class_mask]

        # plot += k3d.points(class_points, point_size=0.1, color=colormap[class_label % len(colormap)])
        print(class_label, cfg.classes[class_label], color_names[class_label])

        if class_label in [cfg.classes.index(c) for c in flat_classes]:
            clusters = segment_planes_and_clusters(class_points, eps=eps, min_samples=min_samples)
            for c in clusters:
                if not too_small(c):
                    segmented_objects.append((c, class_label))
        
        else:
            # Perform DBSCAN clustering to segment individual objects
            clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(class_points)
            cluster_labels = clustering.labels_  # -1 means noise, rest are cluster indices

            unique_clusters = np.unique(cluster_labels[cluster_labels != -1])  # Ignore noise (-1)
            for cluster_id in unique_clusters:
                object_mask = cluster_labels == cluster_id
                object_points = class_points[object_mask]
                if not too_small(object_points):
                    segmented_objects.append((object_points, class_label))

    return segmented_objects

def segment_planes_and_clusters(point_cloud, ransac_threshold=0.125, min_plane_size=100, eps=0.1, min_samples=10):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)

    clusters = []
    remaining_pcd = pcd

    while True:
        # Step 1: Identify Plane
        if np.asarray(remaining_pcd.points).shape[0] < min_plane_size:
            break

        plane_model, inliers = remaining_pcd.segment_plane(distance_threshold=ransac_threshold,
                                                            ransac_n=3,
                                                            num_iterations=1000)

        if len(inliers) < min_plane_size:
            break  # Stop if no significant planes remain
        
        plane_pcd = remaining_pcd.select_by_index(inliers)
        plane_points = np.asarray(plane_pcd.points)

        # Step 2: Cluster the plane using DBSCAN
        clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(plane_points)
        cluster_labels = clustering.labels_

        unique_clusters = np.unique(cluster_labels[cluster_labels != -1])  # Ignore noise (-1)
        clustered_planes = []
        for cluster_id in unique_clusters:
            cluster_mask = cluster_labels == cluster_id
            cluster_points = plane_points[cluster_mask]
            clustered_planes.append(cluster_points)

        # Store the plane and its clusters
        clusters.extend(clustered_planes)

        # Remove the detected plane from the remaining points
        remaining_pcd = remaining_pcd.select_by_index(inliers, invert=True)

    return clusters

pred_np = pred.cpu().numpy()
objects = segment_individual(coord, pred_np)

# Example output
for i, (obj, class_label) in enumerate(objects[:5]):
    print(f"Object {i}: {obj.shape} points, Class {cfg.classes[class_label]}")

print(len(objects))

[0, 1, 2, 3, 6, 7]
[ 0  1  2  3  6  7  8 10 11 12]
0 ceiling Red
1 floor Green
2 wall Blue
3 beam Yellow
6 door White
7 chair Orange
8 table Purple
10 sofa Lime Green
11 board Pink
12 clutter Gray
Object 0: (212635, 3) points, Class ceiling
Object 1: (190228, 3) points, Class floor
Object 2: (114729, 3) points, Class wall
Object 3: (113164, 3) points, Class wall
Object 4: (72760, 3) points, Class wall
38


In [6]:
# import k3d

# print(len(objects))

colormap = [
    0xFF0000, 0x00FF00, 0x0000FF, 0xFFFF00, 0xFF00FF, 0x00FFFF, 0xFFFFFF, 
    0xFF8000, 0x8000FF, 0x0080FF, 0x80FF00, 0xFF0080, 0x808080,
    0xC0C0C0, 0x800000, 0x808000, 0x008000, 0x800080, 0x008080, 0x000080,
    0xFFD700, 0xFF4500, 0xDA70D6, 0xADFF2F, 0x32CD32, 0x8A2BE2, 0xDC143C,
    0x7FFF00, 0x00FA9A, 0xB22222, 0xD2691E, 0xFF1493, 0xFF69B4, 0x4B0082,
    0x9932CC, 0x8B0000, 0x556B2F, 0x1E90FF, 0x00CED1, 0x48D1CC, 0xBDB76B,
    0x9ACD32, 0x20B2AA, 0x9370DB, 0x5F9EA0, 0x40E0D0, 0xE9967A, 0xF08080, 
    0xE6E6FA, 0xA52A2A, 0x87CEEB
]

plot = k3d.plot()

for i, (obj, class_label) in enumerate(objects):
    plot += k3d.points(obj, point_size=0.1, color=colormap[i % len(colormap)])

plot.display()

Output()

In [7]:
plot = k3d.plot()
# plot.display()

flats = [False for i in range(len(objects))]

for i, (obj, class_label) in enumerate(objects):
    bounds = get_point_cloud_bounds(obj)
    min_width = min([abs(b[1] - b[0]) for b in bounds])
    # print(cfg.classes[class_label], min_width)

    if min_width < 0.2 or cfg.classes[class_label] in flat_classes:
        flats[i] = True
        
        plot += k3d.points(obj, point_size=0.1, color=colormap[i % len(colormap)])
    else:
        print("is not flat:", i, cfg.classes[class_label])

is not flat: 12 table
is not flat: 13 table
is not flat: 14 table
is not flat: 15 table
is not flat: 16 table
is not flat: 17 table
is not flat: 18 table
is not flat: 19 table
is not flat: 20 table
is not flat: 21 table
is not flat: 22 table
is not flat: 23 table
is not flat: 24 table
is not flat: 25 table
is not flat: 26 table
is not flat: 27 sofa
is not flat: 31 clutter
is not flat: 32 clutter
is not flat: 33 clutter
is not flat: 34 clutter


In [8]:
import numpy as np

def generate_flat_interface(bounds, num_points=1000):
    (xmin, xmax), (ymin, ymax), (zmin, zmax) = bounds
    areas = [
        (xmax - xmin) * (ymax - ymin),  # Bottom (zmin)
        (xmax - xmin) * (ymax - ymin),  # Top (zmax)
        (xmax - xmin) * (zmax - zmin),  # Front (ymin)
        (xmax - xmin) * (zmax - zmin),  # Back (ymax)
        (ymax - ymin) * (zmax - zmin),  # Left (xmin)
        (ymax - ymin) * (zmax - zmin),  # Right (xmax)
    ]

    total_area = sum(areas)
    face_ratios = np.array(areas) / total_area  # Normalize to sum to 1
    points_per_face = np.round(face_ratios * num_points).astype(int)

    points = []
    normals = []

    def sample_face(x_range, y_range, z_fixed, count, normal, axis='z'):
        x_samples = np.random.uniform(*x_range, count)
        y_samples = np.random.uniform(*y_range, count)
        z_samples = np.full(count, z_fixed)
        
        if axis == 'x':
            face_points = np.column_stack([z_samples, x_samples, y_samples])  # Swap axes
        elif axis == 'y':
            face_points = np.column_stack([x_samples, z_samples, y_samples])  # Swap axes
        else:
            face_points = np.column_stack([x_samples, y_samples, z_samples])
        
        face_normals = np.tile(normal, (count, 1))  # Assign normal to all points
        return face_points, face_normals

    p, n = sample_face([xmin, xmax], [ymin, ymax], zmin, points_per_face[0], [0, 0, -1])  # Bottom
    points.append(p); normals.append(n)

    p, n = sample_face([xmin, xmax], [ymin, ymax], zmax, points_per_face[1], [0, 0, 1])  # Top
    points.append(p); normals.append(n)

    p, n = sample_face([xmin, xmax], [zmin, zmax], ymin, points_per_face[2], [0, -1, 0], 'y')  # Front
    points.append(p); normals.append(n)

    p, n = sample_face([xmin, xmax], [zmin, zmax], ymax, points_per_face[3], [0, 1, 0], 'y')  # Back
    points.append(p); normals.append(n)

    p, n = sample_face([ymin, ymax], [zmin, zmax], xmin, points_per_face[4], [-1, 0, 0], 'x')  # Left
    points.append(p); normals.append(n)

    p, n = sample_face([ymin, ymax], [zmin, zmax], xmax, points_per_face[5], [1, 0, 0], 'x')  # Right
    points.append(p); normals.append(n)

    return np.vstack(points), np.vstack(normals)

def generate_outer(points, num_samples=100000, expansion_factor=1.05, delta=5000, expansion_size=5):
    hull = ConvexHull(points)
    hull_vertices = points[hull.vertices]

    centroid = np.mean(points, axis=0)
    print("C", centroid)

    # Compute normal vectors (approximate outward directions)
    normals = hull_vertices - centroid
    normals /= np.linalg.norm(normals, axis=1, keepdims=True)  # Normalize

    expanded_hull_points = hull_vertices + expansion_factor * normals

    min_bounds = points.min(axis=0)
    max_bounds = points.max(axis=0)

    ret_pts = np.array([])
    while num_samples >= ret_pts.shape[0]:
        random_points = np.random.uniform(min_bounds - expansion_size, max_bounds + expansion_size, (delta, 3))
        delaunay = Delaunay(hull_vertices)
        outside_mask = ~delaunay.find_simplex(random_points) >= 0  # True if outside

        if ret_pts.shape[0] == 0:
            ret_pts = random_points[outside_mask].copy()
        else:
            ret_pts = np.vstack([ret_pts, random_points[outside_mask].copy()])

    return ret_pts

# Example usage
N = 1000
point_cloud = np.random.rand(N, 3) * 10  # Simulated random Nx3 point cloud

planes_with_clusters = segment_planes_and_clusters(point_cloud)

# Example output
for i, (plane, clusters) in enumerate(planes_with_clusters):
    print(f"Plane {i}: {plane.shape} points, {len(clusters)} clusters")


# Example usage
bounds = [[0, 10], [0, 5], [0, 3]]  # [xmin, xmax], [ymin, ymax], [zmin, zmax]
num_points = 25000  # Total number of points

prism, prism_normals = generate_flat_interface(bounds, num_points)
outer = generate_outer(prism, num_points)
print(prism.shape)

# plot = k3d.plot()        
# plot += k3d.points(prism, point_size=0.1)
# plot += k3d.vectors(prism, prism_normals, line_width=0.01, head_size=0.5, color=0x800000)
# plot += k3d.points(outer, point_size=0.1, color=0x00FF00)
# plot.display()


C [5.02145241 2.48906126 1.50429787]
(25000, 3)


In [12]:
def calculate_bounds(points):
    x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
    y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])
    z_min, z_max = np.min(points[:, 2]), np.max(points[:, 2])
    
    return np.array([[x_min, x_max], [y_min, y_max], [z_min, z_max]]), (np.array([x_min, y_min, z_min]), np.array([x_max, y_max, z_max]))

def time_function(func):
    """Utility to time any function."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.4f} seconds")
        return result
    return wrapper

def generate_thickened_points_within_cone(points, normals, N=1000, K=8, d=0.01, max_angle=np.pi/3):
    """
    For each sampled point, generate K new points within a cone of `max_angle` around its normal.
    The K directions have deviation angles (theta) evenly spaced between 0 and max_angle, 
    and each is assigned an azimuth angle (phi) evenly spaced around the circle.
    This guarantees that one direction (with theta=0) is exactly the normal.
    """
    idx = np.random.choice(len(points), min(N, len(points)), replace=False)
    sampled_points = points[idx]
    sampled_normals = normals[idx]

    # Step 1: Generate an orthonormal basis (normal, tangent1, tangent2) for each sampled point.
    ref = np.array([1.0, 0.0, 0.0], dtype=np.float32)
    tangent1 = np.cross(sampled_normals, ref[None, :])
    degenerate = np.linalg.norm(tangent1, axis=1) < 1e-6
    if np.any(degenerate):
        alt_ref = np.array([0.0, 1.0, 0.0], dtype=np.float32)
        tangent1[degenerate] = np.cross(sampled_normals[degenerate], alt_ref)
    tangent1 /= np.linalg.norm(tangent1, axis=1, keepdims=True)
    tangent2 = np.cross(sampled_normals, tangent1)

    # Step 2: Build direction vectors using a range of deviation angles.
    # Create K deviation angles (theta) evenly spaced between 0 and max_angle.
    thetas = np.linspace(0, max_angle, K)  # deviation angles
    # Create K azimuth angles (phi) evenly spaced from 0 to 2*pi.
    phis = np.linspace(0, 2 * np.pi, K, endpoint=False)
    
    # Compute cosine and sine for the deviation angles.
    cos_thetas = np.cos(thetas)  # shape: (K,)
    sin_thetas = np.sin(thetas)  # shape: (K,)

    # Now, for each sampled normal, construct K directions.
    # For each direction:
    #    direction = cos(theta)*normal + sin(theta)*(cos(phi)*tangent1 + sin(phi)*tangent2)
    # Using broadcasting to get an array of shape (N, K, 3)
    directions = (sampled_normals[:, None, :] * cos_thetas[None, :, None] +
                  (tangent1[:, None, :] * np.cos(phis)[None, :, None] +
                   tangent2[:, None, :] * np.sin(phis)[None, :, None]) * sin_thetas[None, :, None])
    
    # Normalize directions (they should be unit vectors by construction, but this guards against numerical issues)
    directions /= np.linalg.norm(directions, axis=-1, keepdims=True)
    
    # Step 3: Displace the sampled points along each of the K directions.
    new_points = sampled_points[:, None, :] + d * directions
    new_points = new_points.reshape(-1, 3)

    return new_points

def estimate_normals(points, k=10):
    nbrs = NearestNeighbors(n_neighbors=k).fit(points)
    _, indices = nbrs.kneighbors(points)
    normals = []
    for i, idx in enumerate(indices):
        neighbors = points[idx]
        cov = np.cov(neighbors.T)
        eigvals, eigvecs = np.linalg.eigh(cov)
        normals.append(eigvecs[:, 0])  # smallest eigenvector
    return np.array(normals)
    
def smooth_by_local_average(points, k=8):
    nbrs = NearestNeighbors(n_neighbors=k).fit(points)
    _, indices = nbrs.kneighbors(points)
    smoothed = np.array([points[neighbors].mean(axis=0) for neighbors in indices])
    return smoothed

@time_function
def iterative_thicken_timed(points, iterations=3, N=10000, K=8, d=0.01, flip_first=False, smooth_intervals=2):
    print(d)
    indices = np.random.choice(points.shape[0], size=min(N, points.shape[0]), replace=False)
    points = points[indices]
    normals = estimate_normals(points)
    new_normals = normals
    if flip_first:
        flip_mask = np.random.rand(new_normals.shape[0]) < 0.5  # 50% chance
        new_normals[flip_mask] *= -1  # flip selected normals
        print(f"Flipped {np.sum(flip_mask)} normals in first iteration")
        
    for i in range(iterations):
        # indices = np.random.choice(points.shape[0], size=min(N, points.shape[0]), replace=False)
        # new_pts_inp = points[indices]
        new_pts_inp = points[-N:]

        new_pts = generate_thickened_points_within_cone(new_pts_inp, new_normals, N=N, K=K, d=d, max_angle=np.pi / 2.1)
        points = np.vstack([points, new_pts])

        if (i+1) % smooth_intervals == 0:
            points = smooth_by_local_average(points, k=K)

        new_normals = estimate_normals(points[-new_pts.shape[0]:])
        normals = np.vstack([normals, new_normals])

    return points, normals

@time_function
def iterative_thicken_along_normals(points, iterations=3, d=0.01, smooth_interval=2, 
                                    flip_first=False, N=None, noise_level=0.15):
    all_points = points.copy()       # cumulative cloud: original + added points
    first_points = points.copy()     # base points from which to sample in the next iteration
    nneigbors = 8
    if N is None:
        N = first_points.shape[0]
    
    for it in range(iterations):
        # Sample from the base (first_points)
        indices = np.random.choice(first_points.shape[0], size=min(N, first_points.shape[0]), replace=False)
        base_points = first_points[indices]
        
        # Estimate normals for the sampled base points.
        normals = estimate_normals(base_points, k=10)
        
        # Add tunable Gaussian noise to normals.
        if noise_level > 0:
            noise = np.random.normal(scale=noise_level, size=normals.shape)
            normals = normals + noise
            normals = normals / np.linalg.norm(normals, axis=1, keepdims=True)
        
        # Optionally flip normals.
        if flip_first:
            flip_mask = np.random.rand(normals.shape[0]) < 0.5
            normals[flip_mask] *= -1
            print(f"Iteration {it}: Flipped {np.sum(flip_mask)} normals")
        
        # Compute new points by displacing base points along the normals.
        # (Here we use a constant displacement per iteration; adjust if you want a different schedule.)
        new_points = base_points + (d / iterations) * (1 + it - it // iterations) * normals
        
        # Append these new points to the cumulative cloud.
        all_points = np.vstack([all_points, new_points])
        
        # Every 'smooth_interval' iterations, smooth the newly generated points and update base points.
        if (it + 1) % smooth_interval == 0:
            smoothed = smooth_by_local_average(all_points[first_points.shape[0]:], k=nneigbors * (1 + it // smooth_interval))
            first_points = smoothed.copy()

    return all_points, first_points

def remove_outliers(point_cloud, method="statistical", nb_neighbors=25, std_ratio=1.5, radius=0.1, min_neighbors=5):
    # Convert to Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)

    if method == "statistical":
        # Statistical Outlier Removal (SOR)
        clean_pcd, inlier_indices = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
    elif method == "radius":
        # Radius Outlier Removal (ROR)
        clean_pcd, inlier_indices = pcd.remove_radius_outlier(nb_points=min_neighbors, radius=radius)
    else:
        raise ValueError("Invalid method. Use 'statistical' or 'radius'.")

    return np.asarray(clean_pcd.points)

@time_function
def thicken_point_cloud_normals_timed(point_cloud, width):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))

    normals = np.asarray(pcd.normals)
    new_points = point_cloud + (normals * width)
    opp_points = point_cloud - (normals * width)
    new_normals = normals
    opp_normals = -normals
    normals_combined = np.vstack([new_normals, opp_normals])
    thickened_cloud = np.vstack([new_points, opp_points])
    return thickened_cloud, normals_combined

def generate_filtered_point_cloud(A, num_points=100000, width=3.0, min_distance=0.1, tree = None):
    # Step 1: Generate new points above A
    A_min, A_max = np.min(A, axis=0) - width / 2, np.max(A, axis=0) + width / 2
    if not tree: tree = cKDTree(A)
    filtered_points = np.array([])

    x_new = np.random.uniform(A_min[0], A_max[0], num_points)
    y_new = np.random.uniform(A_min[1], A_max[1], num_points)
    z_new = np.random.uniform(A_min[2], A_max[2], num_points) 
    new_points = np.column_stack((x_new, y_new, z_new))

    distances, _ = tree.query(new_points)
    filtered_points = new_points[distances > min_distance]
        
    return filtered_points

def filtered_point_cloud_indices(A, B, min_distance=0.1):
    tree = cKDTree(A)
    distances, _ = tree.query(B)
    print("DIST", distances.mean())
    # filtered_points = B[distances > min_distance]
        
    return distances > min_distance

@time_function
def generate_filtered_point_cloud_timed(A, width, num_points=100000, min_distance=0.1, tree=None):
    A_min, A_max = np.min(A, axis=0) - width / 2, np.max(A, axis=0) + width / 2
    if not tree:
        tree = cKDTree(A)

    x_new = np.random.uniform(A_min[0], A_max[0], num_points)
    y_new = np.random.uniform(A_min[1], A_max[1], num_points)
    z_new = np.random.uniform(A_min[2], A_max[2], num_points)
    new_points = np.column_stack((x_new, y_new, z_new))
    distances, _ = tree.query(new_points)
    filtered_points = new_points[distances > min_distance]
    return filtered_points

@time_function
def sample_states_and_controls_timed(inner_point_cloud, point_cloud, N, K, min_dot=0.25):
    def sample_vector():
        vec = np.random.randn(3)
        return vec / np.linalg.norm(vec)

    idx = np.random.choice(point_cloud.shape[0], N, replace=False)
    sampled_points = point_cloud[idx]
    kdtree = cKDTree(inner_point_cloud)
    distances, nearest_indices = kdtree.query(sampled_points, k=2)
    nearest_neighbors = inner_point_cloud[nearest_indices[:, 1]]
    sampled_data = []

    for i in range(N):
        x, y, z = sampled_points[i]
        uc = nearest_neighbors[i] - sampled_points[i]
        uc = uc / np.linalg.norm(uc)
        sampled_data.append([x, y, z, uc[0], uc[1], uc[2]])
        for _ in range(K - 1):
            random_vector = sample_vector()
            while np.dot(random_vector, uc) < min_dot:
                random_vector = -random_vector
                if np.dot(random_vector, uc) < min_dot:
                    random_vector = sample_vector()
            sampled_data.append([x, y, z, random_vector[0], random_vector[1], random_vector[2]])

    return np.array(sampled_data)

import numpy as np
from scipy.spatial import cKDTree

def average_nearest_neighbor_distance(points):
    tree = cKDTree(points)
    distances, _ = tree.query(points, k=2)
    nn_distances = distances[:, 1]
    return np.mean(nn_distances), np.std(nn_distances)

import numpy as np
from skimage.measure import marching_cubes
import open3d as o3d
from scipy.ndimage import gaussian_filter

def marching_cubes_surface(points: np.ndarray, voxel_size=0.05, padding=5, smoothing=1.0, debug=True):
    """
    Extract surface points using Marching Cubes from a raw point cloud.
    Returns mesh vertices and faces.
    """
    points = np.asarray(points)

    # Step 1: Shift to origin with padding
    min_bound = points.min(axis=0) - voxel_size * padding
    shifted = points - min_bound

    # Step 2: Create 3D voxel grid and accumulate counts
    grid_shape = np.ceil((shifted.max(axis=0) / voxel_size)).astype(int) + padding
    grid = np.zeros(grid_shape, dtype=np.float32)

    # Convert to voxel indices
    indices = np.floor(shifted / voxel_size).astype(int)
    indices = indices[(indices >= 0).all(axis=1) & (indices < grid.shape).all(axis=1)]  # bounds check
    for idx in indices:
        grid[tuple(idx)] += 1.0
        
    # Step 3: Smooth occupancy
    grid = gaussian_filter(grid, sigma=smoothing)

    # Step 4: Run Marching Cubes
    verts, faces, normals, values = marching_cubes(grid, level=0.5)

    # Step 5: Convert from voxel to world coordinates
    verts *= voxel_size  # scale
    verts += min_bound   # shift

    if debug:
        print(f"[DEBUG] Mesh vertex bounds: min {verts.min(axis=0)}, max {verts.max(axis=0)}")

    return verts, faces


def mesh_from_marching_cubes(verts, faces) -> o3d.geometry.TriangleMesh:
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(verts)
    mesh.triangles = o3d.utility.Vector3iVector(faces)
    mesh.compute_vertex_normals()
    return mesh

def sample_surface_points_from_marching_cubes(points: np.ndarray, voxel_size=0.05, num_samples=5000):
    verts, faces = marching_cubes_surface(points, voxel_size)
    tri_mesh = trimesh.Trimesh(vertices=verts, faces=faces, process=False)
    sampled_points, face_indices = trimesh.sample.sample_surface(tri_mesh, num_samples)
    # sampled_normals = tri_mesh.face_normals[face_indices]
    sampled_normals = estimate_normals(sampled_points, k=15)
    mesh = mesh_from_marching_cubes(verts, faces)
    return sampled_points, sampled_normals, mesh


In [30]:
from concurrent.futures import ProcessPoolExecutor

augemented_points = []
n1 = 0
n2 = 100

for kraw, (obj, class_label) in enumerate(objects[n1:n2]):
    k = n1 + kraw
    total_class_start = time.time()
    print(f"\nProcessing class {k}: {cfg.classes[class_label]}, Flat:", flats[k], obj.shape)

    if flats[k]:
        domain_width = 0.15
        smooth_intervals = 3
    else:
        domain_width = 0.075
        smooth_intervals = 3
    
    def compute_interface(int_step):
        interface_width = dm + dstd * int_step
        interface, _, _ = sample_surface_points_from_marching_cubes(domain, voxel_size=interface_width, num_samples=int_samples)
        return interface

    iterations = 6
    min_dst_std = 2
    min_neighbors=10
    
    # Filtering
    domain, outer_domain = iterative_thicken_along_normals(obj, N=min(25000, obj.shape[0]), iterations=iterations, d=domain_width, smooth_interval=smooth_intervals, flip_first=True)
    dm, dstd = average_nearest_neighbor_distance(domain)
    domain_density = dm + dstd * min_dst_std
    print("domain", domain.shape, "dist", domain_density)
    domain = remove_outliers(domain, radius=domain_density, min_neighbors=min_neighbors)

    interface_width = dm + dstd * 2.5
    pts_on_env_width = dm + dstd * 2.25
    
    int_samples = int(min(30000, obj.shape[0] * 1.5))
    arrays = []
    with ProcessPoolExecutor() as executor:
        arrays = list(executor.map(compute_interface, np.linspace(2.5, 3.5, 4)))
    interface_tmp_big = np.concatenate(arrays)

    domain = domain[
        filtered_point_cloud_indices(interface_tmp_big, domain, interface_width)
    ]
    domain = remove_outliers(domain, radius=domain_density, min_neighbors=min_neighbors)

    with ProcessPoolExecutor() as executor:
        arrays1 = list(executor.map(compute_interface, np.linspace(0.7, 2.25, 4)))
    arrays.extend(arrays1)
    interface_tmp_big = np.concatenate(arrays)

    # interface_width = dm + dstd * 1.5
    # interface, normals, _  = sample_surface_points_from_marching_cubes(domain, voxel_size=interface_width, num_samples=min(50000, obj.shape[0] * 3))
    # domain = domain[
    #     filtered_point_cloud_indices(np.vstack([interface, interface_tmp]), domain, interface_width)
    # ]
    # print("domain", domain.shape)

    # pts_on_env_width = dm + dstd * 1.5
    # interface_tmp, _ = iterative_thicken_along_normals(interface, N=min(8000, interface.shape[0]), iterations=7, d=interface_width*1.5, smooth_interval=3, flip_first=True)
    # print("interface_tmp", interface_tmp.shape)
    # pts_on_env, _, _  = sample_surface_points_from_marching_cubes(np.vstack([domain, interface_tmp, interface]), voxel_size=pts_on_env_width, num_samples=min(75000, int(obj.shape[0] * 2.5)))
    # pts_on_env = pts_on_env[
    #     filtered_point_cloud_indices(np.vstack([domain, interface]), pts_on_env, pts_on_env_width)
    # ]
    # pts_on_env_tmp, _ = iterative_thicken_along_normals(pts_on_env, N=min(8000, interface.shape[0]), iterations=10, d=(interface_width + pts_on_env_width), smooth_interval=20, flip_first=True)
    # domain = domain[
    #     filtered_point_cloud_indices(pts_on_env_tmp, domain, interface_width)
    # ]
    
    # post filtering

    domain = remove_outliers(domain, radius=min_dst_std, min_neighbors=min_neighbors)
    interface, normals, _  = sample_surface_points_from_marching_cubes(np.vstack([obj, domain]), voxel_size=interface_width, num_samples=min(50000, domain.shape[0] * 2))
    # interface_tmp, _ = iterative_thicken_along_normals(interface, N=min(8000, interface.shape[0]), iterations=4, d=interface_width, smooth_interval=3, flip_first=True)
    pts_on_env, _, _  = sample_surface_points_from_marching_cubes(np.vstack([domain, interface_tmp_big, interface]), voxel_size=pts_on_env_width, num_samples=min(75000, int(domain.shape[0] * 3)))
    pts_on_env = pts_on_env[
        filtered_point_cloud_indices(np.vstack([domain, interface, interface_tmp_big]), pts_on_env, pts_on_env_width - 1e-1)
    ]
    obj = obj[
        filtered_point_cloud_indices(np.vstack([interface, interface_tmp_big]), obj, interface_width - 1e-1)
    ]
    obj = remove_outliers(obj, radius=domain_density, min_neighbors=min_neighbors)
    domain = np.vstack([domain, obj])

    # pts_on_env, _ = sample_outer_layer_points(domain, offset=0.025,)
    print("interface", interface.shape) 
    print("pts_on_env", pts_on_env.shape)

    env_width = dm + dstd * 50
    min_env_distance = dm + dstd * 5
    outer_width = domain_width + 3.5
    env_num = min(400000, domain.shape[0] * 40)
    envelope = generate_filtered_point_cloud_timed(
        np.vstack([pts_on_env, interface, interface_tmp_big, domain]), width=env_width, num_points=env_num, min_distance=min_env_distance
    )
    print("envelope", envelope.shape)

    envelope_min, envelope_max = np.min(envelope, axis=0), np.max(envelope, axis=0)
    env_center = (envelope_min + envelope_max) / 2
    print("Envelope center:", env_center)

    outers = envelope.copy()  # Placeholder for now

    N = 2000
    K = 40

    control_outs_env = sample_states_and_controls_timed(domain, envelope, N // 3, K)
    control_outs_on_env = sample_states_and_controls_timed(domain, pts_on_env, N // 3, K)
    control_outs_interface = sample_states_and_controls_timed(domain, interface, N // 3, K)
    control_outs = np.vstack([control_outs_env, control_outs_interface, control_outs_on_env])
    control_points, controls = control_outs[:, :3], control_outs[:, 3:]

    all_points = np.vstack([domain, interface, pts_on_env, envelope])
    bounds_og, bounds_coords = calculate_bounds(all_points)
    bbox_min, bbox_max = bounds_coords
    bounds = bounds_og.copy()

    all_points_obj = np.vstack([domain, interface])
    bounds_obj, _ = calculate_bounds(all_points_obj)

    center_for_translation = (bbox_max + bbox_min) / 2
    scale_factor = max(bbox_max - bbox_min) / 2

    print("Bounds:", bounds)
    print("Center for translation:", center_for_translation)
    print("Scale factor:", scale_factor)

    augemented_points.append({
        "class": cfg.classes[class_label],
        "pts_inside": domain,
        "env_outside_pts": envelope,
        "pts_on_env": pts_on_env,
        "outside_points": outers,
        "control_points": control_outs,
        "control_points_on_env": control_outs_on_env,
        "control_points_env": control_outs_env,
        "control_points_interface": control_outs_interface,
        "original": obj,
        "interface_pts": interface,
        "interface_normals": normals,
        "bounds": bounds,
        "bounds_obj": bounds_obj,
        "scale_factor": scale_factor,
        "center_for_translation": center_for_translation
    })

    print(f"Total time for class {cfg.classes[class_label]}: {time.time() - total_class_start:.2f} seconds")



Processing class 0: ceiling, Flat: True (212635, 3)
Iteration 0: Flipped 12532 normals
Iteration 1: Flipped 12540 normals
Iteration 2: Flipped 12689 normals
Iteration 3: Flipped 12509 normals
Iteration 4: Flipped 12584 normals
Iteration 5: Flipped 12526 normals
iterative_thicken_along_normals took 20.6825 seconds
domain (362635, 3) dist 0.029971335718458517
[DEBUG] Mesh vertex bounds: min [-20.651337   36.678993    2.9073532], max [-15.668875   41.331093    3.3761752]
[DEBUG] Mesh vertex bounds: min [-20.631819   36.709896    2.9242733], max [-15.684235   41.31357     3.3670588]
[DEBUG] Mesh vertex bounds: min [-20.619547   36.720253    2.9380023], max [-15.6911545  41.3106      3.3519537]
[DEBUG] Mesh vertex bounds: min [-20.6414    36.687897   2.916464], max [-15.678943   41.32409     3.3750768]
DIST 0.08188115122226498
[DEBUG] Mesh vertex bounds: min [-20.615435   36.72224     3.0169685], max [-15.707883   41.303585    3.2791822]
[DEBUG] Mesh vertex bounds: min [-20.567236   36.771

OSError: [Errno 12] Cannot allocate memory

In [29]:
import k3d
import numpy as np
import random

plot = k3d.plot()

colors = {
    # "original": 0xff0000,             # Red
    "pts_inside": 0x00ff00,           # Green
    "interface_pts": 0x0000ff,        # Blue
    # "env_outside_pts": 0xffff00,      # Yellow
    "pts_on_env": 0xff00ff,           # Magenta
    # "outside_points": 0x00ffff,       # Cyan
    # "control_points": 0x808080,       # Gray
    # "control_points_on_env": 0xFFA500,# Orange
    # "control_points_env": 0x800080,   # Purple
    # "control_points_interface": 0x008000 # Dark Green
}

# print(domain.shape, domain_normals.shape)
# # plot += k3d.vectors(domain[:domain_normals.shape[0]], domain_normals, color=0x0000ff, head_size=0.1)  # Blue
# plot += k3d.vectors(interface, normals, color=0xFFA500, head_size=0.1, line_width=0.001)  # Blue
# plot += k3d.points(interface_tmp, point_size=0.01, color=0x808080)
# plot += k3d.points(pts_on_env_tmp, point_size=0.01, color=0x008000)

# plot += k3d.points(bounds_obj.T, point_size=0.05, color=0x008000)
plot.display()
for i, data_dict in enumerate(augemented_points):
    print(f"Plotting object {i}: {data_dict.get('class', 'Unknown')}")
    
    for key, value in data_dict.items():
        if not isinstance(value, np.ndarray):
            continue
        
        if value.ndim == 2 and value.shape[1] >= 3:
            pts = value[:, :3]
            col = colors.get(key, -1)
            if col != -1:
                plot += k3d.points(pts, point_size=0.005, color=col)
                print(f"{key} bounds: min {pts.min(axis=0)}, max {pts.max(axis=0)}")
                print("ADDED", key, pts.shape)

# plot += k3d.points(interface_tmp_big, point_size=0.01, color=0x008000)


Output()

Plotting object 0: ceiling
pts_inside bounds: min [-20.54584449  36.79217924   3.08222376], max [-15.73660445  41.27041667   3.25505417]
ADDED pts_inside (441577, 3)
pts_on_env bounds: min [-20.69834877  36.6340068    2.8958582 ], max [-15.65463051  41.35468687   3.37059658]
ADDED pts_on_env (75000, 3)
interface_pts bounds: min [-20.62805516  36.70858218   3.0186611 ], max [-15.69571106  41.31345816   3.28519476]
ADDED interface_pts (50000, 3)


In [28]:
# import shutil

# out_folder_name = data_path.split("/")[-1].split(".")[0]
# out_path = os.path.join(
#     "/scratch/rhm4nj/cral/cral-ginn/ginn/myvis/data_gen", 
#     "S3D",
#     out_folder_name
# )

# if not os.path.exists(out_path):
#     os.makedirs(out_path)
# else:
#     shutil.rmtree(out_path)

# skips_names = []

# for idx, values in enumerate(augemented_points):
#     folder_name = f"{idx}_{values['class']}"
#     folder_path = os.path.join(out_path, folder_name)
#     os.makedirs(folder_path)

#     for name, arrays in values.items():
#         if name in skips_names: continue

#         print(f'Saving to {folder_name}:', name)
#         np.save(f'{folder_path}/{name}.npy', arrays)